In [ ]:
# default_exp species_distribution_maps

In [ ]:
import requests as req
import glob
import json

import pandas as pd

In [ ]:
# export

def extract_distribution_map(g_query_response):
    data = g_query_response['data']
    search_taxon = data['searchTaxon']
    num_taxa = search_taxon['totalCount']
    
    edges = search_taxon['edges']
    nodes = []
    
    for edge in edges:
        node = edge['node']
        node['mapasDistribucion'] = node['mapasDistribucion']['edges']
        nodes.append(node)

    return nodes

In [ ]:
cactus_dir = '/Users/RaulSierra/Ecoinformatica/Proyectos/Analisis/OutliersEjemplaresSNIB/snib-datos-geo-validacion/cactus/'
cactus_files = glob.glob(cactus_dir + '/*.csv')

In [ ]:
anfibios_dir = '/Users/RaulSierra/Ecoinformatica/Proyectos/Analisis/OutliersEjemplaresSNIB/snib-datos-geo-validacion/anfibios/'
anfibios_files = glob.glob(anfibios_dir + '/*.csv')

csv_files = cactus_files + anfibios_files

In [ ]:
dfs_list = []
for file in csv_files:
    dfs_list.append(pd.read_csv(file, index_col=0, encoding='latin_1', low_memory=False))

records = pd.concat(dfs_list, sort=False)

In [ ]:
records.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175889 entries, 0 to 111
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   especievalida         175889 non-null  object 
 1   gridid                175889 non-null  int64  
 2   longitud              175889 non-null  float64
 3   latitud               175889 non-null  float64
 4   cuarentena            293 non-null     object 
 5   isoutlier             175889 non-null  bool   
 6   ms_cluster            175889 non-null  int64  
 7   tscore                175889 non-null  float64
 8   clasevalida           175889 non-null  object 
 9   ordenvalido           175889 non-null  object 
 10  familiavalida         175889 non-null  object 
 11  altitudmapa           175238 non-null  float64
 12  ambiente              175889 non-null  object 
 13  aniocolecta           175889 non-null  int64  
 14  probablelocnodecampo  8592 non-null    object 
 15  ver

In [ ]:
records[['especievalida', 'DistConocida', 'DistPotencial']].groupbyby_by(['especievalida', 'DistConocida', 'DistPotencial'])

AttributeError: 'DataFrame' object has no attribute 'group_by'

In [ ]:
species_names = records.especievalida.unique()
len(species_names)

819

In [ ]:
query_template = Template('''
    { searchTaxon(q:"categoria:especie AND taxon:'$genus_var AND $epithet_var'") {
            totalCount
            edges {
              node {
                id
                taxon
                categoria
                mapasDistribucion {
                  totalCount
                  edges {
                    node {
                      gridcode
                      geom
                    }
                  }
                }
              }
            }
          }
        }
''')

In [ ]:
records['existe_mapa_distribucion'] = None
species_results = []
for name in species_names:
    variables = {
       'genus_var' : name.split()[0],
       'epithet_var' : name.split()[1]
    }
    species_result = variables
    
    query = query_template.substitute(variables)
    json_q = { 'query': query }
    r = req.post(conabio_gql, json=json_q)
    res = json.loads(r.text)
    mapas = extract_distribution_map(res)
    species_result['mapas'] = mapas
    

In [ ]:
records.loc[records.existe_mapa_distribucion].especievalida.unique()

array(['Pereskia lychnidiflora'], dtype=object)

In [ ]:
records.query('DistConocida == No')